En esta lección, dirigimos nuestra atención a una prueba estadística común para variables categóricas: la prueba de chi-cuadrado.

# Test Chi-cuadrado bondad de ajuste

En nuestro estudio de las pruebas t, presentamos la prueba t unidireccional para verificar si la media de una muestra difiere de la media esperada (población). La prueba de bondad de ajuste chi-cuadrado es análoga a la prueba t unidireccional para variables categóricas: prueba si la distribución de datos categóricos de muestra coincide con una distribución esperada. Por ejemplo, podría usar una prueba de bondad de ajuste de chi-cuadrado para verificar si la demografía racial del subconjunto de los votantes coincide con la de todo EE. UU.

Cuando se trabaja con datos categóricos, los valores en sí mismos no son de mucha utilidad para las pruebas estadísticas porque categorías como "masculino", "femenino" y "otro" no tienen significado matemático. Las pruebas que tratan con variables categóricas se basan en recuentos de variables en lugar del valor real de las propias variables.

Generemos algunos datos demográficos falsos para EE. UU. y Minnesota y realizaremos la prueba de bondad de ajuste chi-cuadrado para verificar si son diferentes:

In [ ]:
import numpy as np
import pandas as pd
import scipy.stats as stats

In [ ]:
national = pd.DataFrame(["white"]*100000 + ["hispanic"]*60000 +\
                        ["black"]*50000 + ["asian"]*15000 + ["other"]*35000)


minnesota = pd.DataFrame(["white"]*600 + ["hispanic"]*300 + \
                         ["black"]*250 +["asian"]*75 + ["other"]*150)

national_table = pd.crosstab(index=national[0], columns="count")
minnesota_table = pd.crosstab(index=minnesota[0], columns="count")

print( "National")
print(national_table)
print(" ")
print( "Minnesota")
print(minnesota_table)

National
col_0      count
0               
asian      15000
black      50000
hispanic   60000
other      35000
white     100000
 
Minnesota
col_0     count
0              
asian        75
black       250
hispanic    300
other       150
white       600


Las pruebas de chi-cuadrado se basan en la llamada estadística de chi-cuadrado. Se calcula la estadística de chi-cuadrado con la siguiente fórmula:



$\tilde{\chi}^2=\sum_{k=1}^{n} \frac{(O_k - E_k)^2}{E_k}$

En la fórmula, observado es el conteo real observado para cada categoría y esperado es el conteo esperado basado en la distribución de la población para la categoría correspondiente. Calculemos la estadística de chi-cuadrado para nuestros datos para ilustrar:

In [ ]:
observed = minnesota_table

national_ratios = national_table/len(national)  # Get population ratios

expected = national_ratios * len(minnesota)   # Get expected counts

chi_squared_stat = (((observed-expected)**2)/expected).sum()

print(chi_squared_stat)

col_0
count    18.194805
dtype: float64


Nota: La prueba de chi-cuadrado supone que ninguno de los recuentos esperados es inferior a 5.

Similar a la prueba t donde comparamos la estadística de prueba t con un valor crítico basado en la distribución t para determinar si el resultado es significativo, en la prueba de chi-cuadrado comparamos la estadística de prueba de chi-cuadrado con un valor crítico basado en la distribución chi-cuadrado. La abreviatura de la biblioteca de Scipy para la distribución de chi-cuadrado es chi2. Usaremos este conocimiento para encontrar el valor crítico para un nivel de confianza del 95 % y verificaremos el valor p de nuestro resultado:

In [ ]:
crit = stats.chi2.ppf(q = 0.95, # Find the critical value for 95% confidence*
                      df = 4)   # Df = number of variable categories - 1

print("Critical value")
print(crit)

p_value = 1 - stats.chi2.cdf(x=chi_squared_stat,  # Find the p-value
                             df=4)
print("P value")
print(p_value)

Critical value
9.487729036781154
P value
[0.00113047]


Dado que nuestra estadística chi-cuadrado excede el valor crítico, rechazaríamos la hipótesis nula de que las dos distribuciones son iguales.

Puede realizar una prueba de bondad de ajuste chi-cuadrado automáticamente usando la función scipy scipy.stats.chisquare():

In [ ]:
stats.chisquare(f_obs= observed,   # Array of observed counts
                f_exp= expected)   # Array of expected counts

Power_divergenceResult(statistic=array([18.19480519]), pvalue=array([0.00113047]))

Los resultados de la prueba concuerdan con los valores que calculamos anteriormente.

#Test Chi-Cuadrado de independencia
La independencia es un concepto clave en probabilidad que describe una situación en la que conocer el valor de una variable no dice nada sobre el valor de otra. Por ejemplo, el mes en que se nace probablemente no tenga nada que versobre qué navegador web usa, por lo que esperaríamos que el mes de nacimiento y la preferencia del navegador sean independientes. Por otro lado, el mes de nacimiento podría estar relacionado con si se destacaen los deportes en la escuela, por lo que el mes de nacimiento y el rendimiento deportivo podrían no ser independientes.

La prueba de independencia chi-cuadrado comprueba si dos variables categóricas son independientes. La prueba de independencia se usa comúnmente para determinar si variables como la educación, las opiniones políticas y otras preferencias varían según factores demográficos como el género, la raza y la religión. Generemos algunos datos de encuestas de votantes falsos y realicemos una prueba de independencia:

In [ ]:
np.random.seed(10)

# Sample data randomly at fixed probabilities
voter_race = np.random.choice(a= ["asian","black","hispanic","other","white"],
                              p = [0.05, 0.15 ,0.25, 0.05, 0.5],
                              size=1000)

# Sample data randomly at fixed probabilities
voter_party = np.random.choice(a= ["democrat","independent","republican"],
                              p = [0.4, 0.2, 0.4],
                              size=1000)

voters = pd.DataFrame({"race":voter_race,
                       "party":voter_party})

voter_tab = pd.crosstab(voters.race, voters.party, margins = True)

voter_tab.columns = ["democrat","independent","republican","row_totals"]

voter_tab.index = ["asian","black","hispanic","other","white","col_totals"]

observed = voter_tab.iloc[0:5,0:3]   # Get table without totals for later use
voter_tab

,democrat,independent,republican,row_totals
asian,21,7,32,60
black,65,25,64,154
hispanic,107,50,94,251
other,15,8,15,38
white,189,96,212,497
col_totals,397,186,417,1000




Para una prueba de independencia, usamos la misma fórmula de chi-cuadrado que usamos para la prueba de bondad de ajuste. La principal diferencia es que tenemos que calcular los recuentos esperados de cada celda en una tabla bidimensional en lugar de una tabla unidimensional. Para obtener el recuento esperado para una celda, se multiplica el total de la fila de esa celda por el total de la columna de esa celda y luego divida por el número total de observaciones. Podemos obtener rápidamente los recuentos esperados para todas las celdas de la tabla tomando los totales de las filas y las columnas de la tabla, realizando un producto externo con la función np.outer() y dividiendo por el número de observaciones:

In [ ]:
expected =  np.outer(voter_tab["row_totals"][0:5],
                     voter_tab.loc["col_totals"][0:3]) / 1000

expected = pd.DataFrame(expected)

expected.columns = ["democrat","independent","republican"]
expected.index = ["asian","black","hispanic","other","white"]

expected

,democrat,independent,republican
asian,23.820,11.160,25.020
black,61.138,28.644,64.218
hispanic,99.647,46.686,104.667
other,15.086,7.068,15.846
white,197.309,92.442,207.249


Ahora podemos seguir los mismos pasos que dimos antes para calcular el estadístico chi-cuadrado, el valor crítico y el valor p:

In [ ]:
chi_squared_stat = (((observed-expected)**2)/expected).sum().sum()

print(chi_squared_stat)

7.169321280162059


Nota: Llamamos a .sum() dos veces: una para obtener las sumas de las columnas y una segunda vez para sumar las sumas de las columnas, devolviendo la suma de toda la tabla 2D.

In [ ]:
crit = stats.chi2.ppf(q = 0.95, # Find the critical value for 95% confidence*
                      df = 8)   # *

print("Critical value")
print(crit)

p_value = 1 - stats.chi2.cdf(x=chi_squared_stat,  # Find the p-value
                             df=8)
print("P value")
print(p_value)

Critical value
15.50731305586545
P value
0.518479392948842


Nota: Los grados de libertad para una prueba de independencia son iguales al producto del número de categorías en cada variable menos 1. En este caso tenemos una tabla de 5x3 por lo que df = 4x2 = 8.

Al igual que con la prueba de bondad de ajuste, podemos usar scipy para realizar una prueba de independencia rápidamente. Use la función stats.chi2_contingency() para realizar una prueba de independencia automáticamente dada una tabla de frecuencia de conteos observados:

In [ ]:
stats.chi2_contingency(observed= observed)

(7.169321280162059, 0.518479392948842, 8, array([[ 23.82 ,  11.16 ,  25.02 ],
        [ 61.138,  28.644,  64.218],
        [ 99.647,  46.686, 104.667],
        [ 15.086,   7.068,  15.846],
        [197.309,  92.442, 207.249]]))

La salida muestra la estadística de chi-cuadrado, el valor p y los grados de libertad seguidos de los recuentos esperados.

Como era de esperar, dado el alto valor de p, el resultado de la prueba no detecta una relación significativa entre las variables.

# Resumen
Las pruebas de chi-cuadrado proporcionan una forma de investigar las diferencias en las distribuciones de variables categóricas con las mismas categorías y la dependencia entre variables categóricas. En la próxima lección, aprenderemos sobre una tercera prueba de inferencia estadística, el análisis de varianza, que nos permite comparar varias medias de muestra al mismo tiempo.